# SSMEO Lab 01 : Image coordinates, distorsion model

***

### Import packages

In [2]:
import numpy as np
import cv2
from scipy.optimize import fsolve

## Part 01 : Image measurements
***

<div class="alert alert-block alert-info">
<b>Objectives:</b> In this section you will familiarize yourlsef with image format, image coordinates and manual measurements on images.
</div>

### Read image, define parameters

In [3]:
im_path = '/media/topo/Data/ALS/Vallet/Images/18704.jpg'
img = cv2.imread(im_path)

out_path = '/media/topo/Data/ALS/Vallet/Images/'

measurements = {'x':[], 'y':[]}

### Define function to record image measurements 

In [24]:
def click_event(event, x, y, flags, measurements):
    buf = 25
    #On double left click :
    if event == cv2.EVENT_LBUTTONDBLCLK:
        #Draw cross at clicked pixel
        cv2.line(img, (x,y-buf), (x,y+buf), (50,90,255), 1)
        cv2.line(img, (x-buf,y), (x+buf,y), (50,90,255), 1)
        cv2.putText(img, f"{x}, {y}",(x+5, y-5),cv2.FONT_HERSHEY_SIMPLEX,1,(50,90,255),2)

        #Update image
        cv2.imshow('Manual measurement', img)
        
        #Store clicked x and y coordinate into measurement dictionnary
        measurements['x'].append(x)
        measurements['y'].append(y)

### Image display loop

In [ ]:
cv2.namedWindow('Manual measurement', cv2.WINDOW_GUI_NORMAL)
cv2.setMouseCallback('Manual measurement', click_event, measurements)
cv2.imshow('Manual measurement', img)
cv2.resizeWindow('Manual measurement', 1200,800)

while True:
    k = cv2.waitKey(100)
    if k == 27: # wait for ESC key to exit
        cv2.destroyAllWindows()
        break        
    if cv2.getWindowProperty('Manual measurement',cv2.WND_PROP_VISIBLE) < 1:# wait for window to be closed        
        break        
cv2.destroyAllWindows()

<div class="alert alert-block alert-info">
Pay attention to the coordinates in pixels when you do a measurement. Where is the origin of X and Y axis ? How are they oriented ?
</div>

### Save measurements to txt file

In [ ]:
xy_array = np.array([measurements['x'], measurements['y']]).T
print("Saved measurements :\n" + str(xy_array))

In [ ]:
np.savetxt(out_path + 'coord.txt', xy_array, fmt='%d')
print(f"Saving {xy_array.shape[0]} measurements to {out_path}coord.txt :")

## Part 02 : Correcting camera distorsion
***

<div class="alert alert-block alert-info">
<b>Objectives:</b> Cameras are not perfect and distorsion due to imperfect mounting and lens distorsion must me corrected to accurately map each pixel. In this section you will implement a simple distorsion model and apply it to your previously measurement points
</div>

### Import back saved measurements

In [3]:
im_path = '/media/topo/Data/ALS/Vallet/Images/18704.jpg'
img = cv2.imread(im_path)

out_path = '/media/topo/Data/ALS/Vallet/Images/'
xy_dist = np.loadtxt(out_path + 'coord.txt', delimiter=' ')
print(xy_dist)

[[7726. 1173.]
 [6513. 1697.]
 [4319. 1367.]
 [3999. 1901.]
 [2106. 1241.]
 [1980. 1823.]
 [3707. 1193.]
 [5775. 1697.]
 [8920.  776.]
 [6192.  843.]]


### Define camera distorsion model

We will use the simplified Contrady-Brown distorsion model. 
This model relates distorded image coordinates $x_d, y_d$ (the points you just measured), to the undistorded ones $x, y$ through radial coefs ($k_1, k_2$) and tangential coefs ($p_1, p_2$) :
    
$x_d = x(k_1*r² + k_2 * r⁴) + p_1(r² + 2*x²) + 2*p_2*x*y$
    
$y_d = y(k_1*r² + k_2 * r⁴) + p_2(r² + 2*y²) + 2*p_1*x*y$
      
$r² = x² + y²$

<div class="alert alert-block alert-info">
Note the in this case, X and Y axis are not defined as in Part 01:    
<ul>
    <li>The origin is the optical center of the image</li>
    <li>X is oriented horizontally →</li>
    <li>Y completes the right hand system ↑</li>
</ul> 
</div>

<div class="alert alert-block alert-danger">
<b>To check</b> Replace distorsion coefficient below with real coeficient and check that the solving work properly on real data 
All coefficient = 0 does not correspond to and undistorded model, right ? Hard to evaluate the correctness of the model without knowing the coef corresponding to an udistorded lens. I need to try with real data and check values    
</div>

In [4]:
# Radial coefficients
k1 = 0.0000000000015
k2 = 0.000000000000015
# Tangential coefficients
p1 = 0.000000001
p2 = 0.000000001

In [5]:
def undistord_measurements(var,x_d,y_d,k1,k2,p1,p2):  
    #Unpack variables
    x, y = var
  
    r2 = x*x + y*y
    
    return [x*(k1*r2 + k2*r2*r2) + p1*(r2+2*x*x) + 2*p2*x*y-x_d,
            y*(k1*r2 + k2*r2*r2) + p2*(r2+2*y*y) + 2*p1*x*y-y_d]

### Change pixel coordinates system :
From opencv's top-left origin, X axis →, Y axis ↓ 

To a system compatible with Contrady Brown model: center origin, X axis →, Y axis ↑

In [6]:
x_center = img.shape[1]//2
y_center = img.shape[0]//2

xy_dist_proj = (xy_dist - np.array([x_center,y_center]))*np.array([1,-1])
print(xy_dist_proj)

[[ 2562.  2707.]
 [ 1349.  2183.]
 [ -845.  2513.]
 [-1165.  1979.]
 [-3058.  2639.]
 [-3184.  2057.]
 [-1457.  2687.]
 [  611.  2183.]
 [ 3756.  3104.]
 [ 1028.  3037.]]


### Apply camera distorsion model

In [7]:
xy_undist = []

for var in zip(xy_dist_proj[:,0],xy_dist_proj[:,1]):
    xy_undist.append(fsolve(undistord_measurements, (var), args=(var[0], var[1],k1,k2,p1,p2)))
 
xy_undist=np.array(xy_undist)
print(xy_undist)

xy_undist_proj = (xy_undist*np.array([1,-1]))+np.array([x_center,y_center])
print(xy_undist_proj)

[[ 2071.36629739  2188.59860347]
 [ 1470.14725072  2379.05007935]
 [ -897.17992483  2668.14781659]
 [-1387.60051944  2357.11022512]
 [-2318.34181325  2000.67480183]
 [-2539.70393727  1640.74522007]
 [-1380.54891993  2545.98628457]
 [  735.31289116  2627.17204906]
 [ 2450.7347862   2025.31322583]
 [  937.49938571  2769.65106293]]
[[7235.36629739 1691.40139653]
 [6634.14725072 1500.94992065]
 [4266.82007517 1211.85218341]
 [3776.39948056 1522.88977488]
 [2845.65818675 1879.32519817]
 [2624.29606273 2239.25477993]
 [3783.45108007 1334.01371543]
 [5899.31289116 1252.82795094]
 [7614.7347862  1854.68677417]
 [6101.49938571 1110.34893707]]


### Plot both distorded and undistorded position of each measurements

In [12]:
img = cv2.imread(im_path)
cv2.namedWindow('Distortion plot', cv2.WINDOW_GUI_NORMAL)
cv2.imshow('Distortion plot', img)
cv2.resizeWindow('Distortion plot', 1200,800)


for var in zip(xy_undist_proj[:,0],xy_undist_proj[:,1],xy_dist[:,0],xy_dist[:,1]):
    x_undist = int(var[0])
    y_undist = int(var[1])
    x_dist = int(var[2])
    y_dist = int(var[3])

    
    cv2.line(img, (x_undist,y_undist-buf), (x_undist,y_undist+buf), (0,255,0), 10)
    cv2.line(img, (x_undist-buf,y_undist), (x_undist+buf,y_undist), (0,255,0), 10)
    
    cv2.line(img, (x_dist,y_dist-buf), (x_dist,y_dist+buf), (0,0,255), 10)
    cv2.line(img, (x_dist-buf,y_dist), (x_dist+buf,y_dist), (0,0,255), 10)
    
    cv2.line(img, (x_undist,y_undist), (x_dist,y_dist), (255,0,0), 10)
    cv2.imshow('Distortion plot', img)
    
while True:
    k = cv2.waitKey(100)
    if k == 27: # wait for ESC key to exit
        cv2.destroyAllWindows()
        break        
    if cv2.getWindowProperty('Distortion plot',cv2.WND_PROP_VISIBLE) < 1:# wait for window to be closed        
        break        
cv2.destroyAllWindows()

In [352]:
xy_undist[:,0]

array([1502.09440348, 2877.14602515, 2920.27795949, 2838.81239175,
        757.06709367, 1522.77783783, 2691.07670112])